In [1]:
import stackstac
import pystac_client

URL = "https://stac.test.maap-project.org"
catalog = pystac_client.Client.open(URL)

stac_items = catalog.search(
    collections=["s1-rtc-seasonal-composite"],
).get_all_items()

len(stac_items)

stack = stackstac.stack(stac_items, epsg=32633)

In [2]:
stack

<xarray.DataArray 'stackstac-b7fa6e5ce96015b2bffd402fe11936fb' (time: 36,
                                                                band: 1,
                                                                y: 221071,
                                                                x: 58126)>
dask.array<fetch_raster_window, shape=(36, 1, 221071, 58126), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time           (time) datetime64[ns] 2023-03-14 2023-03-14 ... 2023-03-14
    id             (time) <U48 'Norway_fid_6_p73_f195_Spring_vv_median_202303...
  * band           (band) <U11 'cog_default'
  * x              (x) float64 -5.901e+05 -5.901e+05 ... 1.154e+06 1.154e+06
  * y              (y) float64 1.297e+07 1.297e+07 ... 6.339e+06 6.339e+06
    proj:epsg      (time) float64 3.263e+04 3.263e+04 ... 3.261e+04 3.261e+04
    proj:geometry  (time) object {'type': 'Polygon', 'coordinates': [[[254250...
    epsg           int64 32633
Attributes:
    spec:        RasterSpec(epsg=32633, bounds=(-590130.0, 6339030.0, 1153650...
    crs:         epsg:32633
    transform:   | 30.00, 0.00,-590130.00|\n| 0.00,-30.00, 12971160.00|\n| 0....
    resolution:  30.0